# verify-c-common Empirical Results
This notebook is an alternative report of empirical results made on a case study paper "Verifying Verified Code".


## Part one: Benckmark Prepration and Setup


|  | CBMC | SeaHorn | SMACK | Symbiotic | KLEE | libfuzzer
| :- |:------------- | :------------- | :------------- | :------------- | :------------- | :-------------
| # Benchmark Covered| 168  | 168 | 159 | 155 | 153 | 78 
| Timeout | 2000 seconds| 300 seconds | 300 seconds | 5000 seconds | -- | --
| Total Running Time | 6476 seconds | 631 seconds | 8525 seconds | 10946 seconds | 8577 seconds | --
| Benchmark Source | [aws-c-common](https://github.com/awslabs/aws-c-common/tree/main/verification/cbmc/proofs) | [verify-c-common](https://github.com/seahorn/verify-c-common/tree/master/seahorn/jobs) | [verify-c-common](https://github.com/seahorn/verify-c-common/tree/master/seahorn/jobs)| [verify-c-common](https://github.com/seahorn/verify-c-common/tree/master/seahorn/jobs) | [verify-c-common](https://github.com/seahorn/verify-c-common/tree/master/seahorn/jobs) | [verify-c-common](https://github.com/seahorn/verify-c-common/tree/master/seahorn/jobs)
| Options | Inidividual: <br/>`make results ` <br/> Unit Test: <br/> a python scripts used to traverse all harnnesses | Inidividual: <br/>`./verify [options] <BC_FILE_NAME> ` <br/> Unit Test: <br/> `ctest -D ExperimentalTest -R . --timeout 300` | Inidividual: <br/>`smack --check assertions --time-limit 300 --no-memory-splitting --integer-encoding unbounded-integer --pointer-encoding unbounded-integer [options] <SMACK_LL_FILE_NAME>` <br/> Unit Test: <br/> `ctest -D ExperimentalTest -R smack_ --timeout 300` | Inidividual: <br/>`symbiotic --replay-error --report=short --prp=assert [options] <SYMBIOTIC_BC_FILE_NAME>` <br/> Unit Test: <br/> `ctest -D ExperimentalTest -R symbiotic_ --timeout 5000` | Inidividual: <br/>`klee --libc=uclibc --exit-on-error <KLEE_BC_FILE_NAME> ` <br/> Unit Test: <br/> `ctest -D ExperimentalTest -R klee_` | --
| Results | [aws-cbmc.csv](https://github.com/seahorn/verify-c-common/tree/master/res/aws-cbmc.csv) | [seahorn.csv](https://github.com/seahorn/verify-c-common/tree/master/res/seahorn.csv) |[smack.csv](https://github.com/seahorn/verify-c-common/tree/master/res/smack.csv) | [symbiotic.csv](https://github.com/seahorn/verify-c-common/tree/master/res/symbiotic.csv) | [klee.csv](https://github.com/seahorn/verify-c-common/tree/master/res/klee.csv) | [Coverage report](https://seahorn.github.io/verify-c-common/fuzzing_coverage/index.html)


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.backends.backend_pdf import PdfPages

# Do not limit number of rows displayed
pd.set_option('display.max_rows', None)

In [2]:
# Include data files
SEAHORN_CSV = 'https://raw.githubusercontent.com/seahorn/verify-c-common/master/res/seahorn.csv'
CBMC_CSV = 'https://raw.githubusercontent.com/seahorn/verify-c-common/master/res/aws-cbmc.csv'
SMACK_CSV = 'https://raw.githubusercontent.com/seahorn/verify-c-common/master/res/smack.csv'
SYMBIOTIC_CSV = 'https://raw.githubusercontent.com/seahorn/verify-c-common/master/res/symbiotic.csv'
KLEE_CSV = 'https://raw.githubusercontent.com/seahorn/verify-c-common/master/res/klee.csv'
LOC_CSV = 'https://raw.githubusercontent.com/seahorn/verify-c-common/master/res/loc.csv'

In [3]:
# The result for normal mode...
seahorn_norm = pd.read_csv(SEAHORN_CSV)
seahorn_norm.loc[seahorn_norm['Timing'] == 'Completed', 'Timing'] = 0.0
seahorn_norm['Timing'] = seahorn_norm['Timing'].astype(float)
# The result for aws/cbmc...
aws_cbmc = pd.read_csv(CBMC_CSV)
# The result for smack...
smack = pd.read_csv(SMACK_CSV)
# The result for symbiotic...
symbiotic = pd.read_csv(SYMBIOTIC_CSV)
# The result for klee...
klee = pd.read_csv(KLEE_CSV)
# The result for loc...
jobs_loc = pd.read_csv(LOC_CSV)

# Rename column's name
cbmc_time = aws_cbmc[['bench name', 'total times (s)']].rename(columns = {'total times (s)': 'CBMC time (s)', 'bench name': 'job name'})
seahorn_time = seahorn_norm[['Name', 'Timing']].rename(columns = {'Timing':'Seahorn normal time (s)'})
smack.loc[smack['Timing'] > 200.0, 'Result'] = 'timeout'
smack_time = smack[['Name', 'Timing', 'Result']].rename(columns = {'Timing':'Smack time (s)', 'Result': 'Smack result'})
symbiotic.loc[symbiotic['Timing'] > 5000.0, 'Result'] = 'timeout'
symbiotic_time = symbiotic[['Name', 'Timing', 'Result']].rename(columns = {'Timing':'Symbiotic time (s)', 'Result': 'Symbiotic result'})
klee_time = klee[['Name', 'Timing']].rename(columns = {'Timing': 'Klee time (s)'})
jobs_loc = jobs_loc.rename(columns = {'LOC': 'loc'})
# Rename job's name
seahorn_time['job name'] = seahorn_time['Name'].map(lambda name : 'aws_{}'.format(name[:-11]))
smack_time['job name'] = smack_time['Name'].map(lambda name : 'aws_{}'.format(name[6:-5]))
symbiotic_time['job name'] = symbiotic_time['Name'].map(lambda name : 'aws_{}'.format(name[10:-5]))
klee_time['job name'] = klee_time['Name'].map(lambda name : 'aws_{}'.format(name[5:-5]))
jobs_loc['job name'] = jobs_loc['Name'].map(lambda name : 'aws_{}'.format(name))
# Drop uncessary columns
seahorn_time = seahorn_time.drop(columns=['Name'])
smack_time = smack_time.drop(columns=['Name'])
symbiotic_time = symbiotic_time.drop(columns=['Name'])
klee_time = klee_time.drop(columns=['Name'])

# Merge results into one dataframe and rearrange it
sea_cbmc = pd.merge(seahorn_time, cbmc_time, how='outer', on='job name')
sea_cbmc_smack = pd.merge(smack_time, sea_cbmc, how='outer', on='job name')
sea_cbmc_smack_symb = pd.merge(symbiotic_time, sea_cbmc_smack, how='outer', on='job name')
sea_cbmc_klee = pd.merge(sea_cbmc_smack_symb, klee_time, how='outer', on='job name')
comb_loc = pd.merge(sea_cbmc_klee, jobs_loc, how='outer', on='job name')
combined = comb_loc[['job name', 'loc', 'CBMC time (s)', 'Seahorn normal time (s)', 'Smack time (s)', 'Smack result', 'Symbiotic time (s)', 'Symbiotic result', 'Klee time (s)']]

# Remove unwanted harnesses res
drop_harness_lst = ['aws_hash_iter_begin_done', 'memcpy_using_uint64', 'memset_override_0', 
                    'memset_using_uint64', 'aws_hash_table_foreach_deep_loose', 
                    'aws_byte_buf_eq2', 'aws_hash_table_foreach_deep_precise']
for drop_harn in drop_harness_lst:
  idx = combined.index[combined['job name'] == drop_harn].tolist()
  combined = combined.drop(idx)
  # combined.drop([combined.loc[combined['job name'] == drop_harn].index[0]], inplace=True)

# Save loc as int
combined.loc[:,'loc'] = combined.loc[:,'loc'].astype(int)

# Reindex
combined.index = np.arange(len(combined.index))

# Output data combined
combined

,job name,loc,CBMC time (s),Seahorn normal time (s),Smack time (s),Smack result,Symbiotic time (s),Symbiotic result,Klee time (s)
0,aws_is_power_of_two,11,3.443,0.444965,7.21547,failed,1.515958,passed,0.718919
1,aws_round_up_to_power_of_two,37,3.459,0.642792,7.41379,failed,4.046049,passed,0.794902
2,aws_ptr_eq,10,3.504,0.500028,4.44428,passed,4.671042,passed,0.727793
3,aws_array_eq_ignore_case,78,5.536,2.080760,200.02300,timeout,6.290933,passed,33.526500
4,aws_array_list_capacity,107,5.530,0.786557,11.60290,passed,5.474102,passed,13.619400
5,aws_array_list_back,129,13.785,0.952469,12.32640,passed,8.467914,passed,90.595000
6,aws_array_list_clean_up,89,5.523,0.498641,10.87460,passed,4.724258,passed,13.826800
7,aws_add_size_saturating,37,3.447,0.549630,5.47448,passed,8.886539,passed,0.811040
8,aws_array_list_clear,77,5.511,1.423880,10.55720,passed,4.150718,passed,13.905400
9,aws_array_eq,100,5.630,2.196110,16.68940,passed,9.633242,passed,34.978200


In [5]:
def job_category(job):
  """
  A function to classify job by given name
  """
  if job.startswith('aws_hash_table'):
    return 'hash_table'
  elif job.startswith('aws_hash_iter'):
    return 'hash_iter'
  elif job.startswith('aws_hash_callback'):
    return 'hash_callback'
  elif job.startswith('aws_hash_'):
    return 'hash_table'
  elif job.startswith('aws_array_list'):
    return 'array_list'
  elif job.startswith('aws_array'):
    return 'array'
  elif job.startswith('aws_byte_buf'):
    return 'byte_buf'
  elif job.startswith('aws_byte_cursor'):
    return 'byte_cursor'
  elif job.startswith('aws_linked_list'):
    return 'linked_list'
  elif job.startswith('aws_string'):
    return 'string'
  elif job.startswith('aws_priority_queue'):
    return 'priority_queue'
  elif job.startswith('aws_ring_buffer'):
    return 'ring_buffer'
  elif job.startswith('mem'):
    return 'cbmc_mem'
  elif 'add' in job or 'mul' in job or 'power_of_two' in job:
    return 'arithmetic'
  else:
    return 'others'

combined.loc[:, 'category'] = combined.loc[:, 'job name'].map(job_category)

In [6]:
# Group data by category
combined_by_category = combined.groupby(by='category')
sum_table_1 = pd.DataFrame(combined.groupby(by='category').sum())
table_1 = pd.DataFrame(combined_by_category.describe(percentiles=[]).round(1))

# Get number of covered jobs for each tool
smack_covered_num = int(table_1['Smack time (s)']['count'].sum())
symbiotic_covered_num = int(table_1['Symbiotic time (s)']['count'].sum())
klee_covered_num = int(table_1['Klee time (s)']['count'].sum())
fuzz_covered_num = int(table_1['loc'].iloc[2:5, 0].sum())
total_num = int(table_1['loc']['count'].sum())
print(f'total number of unit proofs: {total_num}\n CBMC:{total_num}, Seahorn:{total_num}, SMACK: {smack_covered_num}, Symbiotic: {symbiotic_covered_num}, klee: {klee_covered_num}, fuzz:{fuzz_covered_num}')

total number of unit proofs: 168
 CBMC:168, Seahorn:168, SMACK: 159, Symbiotic: 160, klee: 153, fuzz:76


In [7]:
# Show the result table
print(f'\t\t\t\t\tTable 1. Verification time results for CBMC, SeaHorn, SMACK, Symbiotic, KLEE and libFuzzer.')
table_1

					Table 1. Verification results for CBMC, SeaHorn, SMACK, Symbiotic, KLEE and libFuzzer.


loc                                   CBMC time (s)         \
               count   mean   std    min    50%    max         count   mean   
category                                                                      
arithmetic       6.0   33.7  11.2   11.0   37.5   40.0           6.0    3.8   
array            4.0   97.5  14.2   78.0  100.0  112.0           4.0    5.6   
array_list      23.0  126.1  24.9   77.0  125.0  181.0          23.0   35.8   
byte_buf        29.0   97.1  33.4   50.0   97.0  188.0          29.0   17.6   
byte_cursor     24.0   98.4  28.7   47.0   97.5  179.0          24.0    6.9   
hash_callback    3.0  115.7  75.7   49.0  100.0  198.0           3.0    9.7   
hash_iter        4.0  177.0   8.7  169.0  177.0  185.0           4.0   12.8   
hash_table      19.0  172.9  77.9   36.0  149.0  328.0          19.0   23.5   
linked_list     18.0  115.7  71.4   17.0  155.0  219.0          18.0   58.9   
others           2.0   15.5   7.8   10.0   15.5   21.0           2.0    3.5   
priority_queue  15.0  187.8  43.4  136.0  171.0  258.0          15.0  208.1   
ring_buffer      6.0  155.7  70.9   56.0  172.5  227.0           6.0   20.0   
string          15.0   87.6  46.4   11.0   90.0  209.0          15.0    6.3   

                            ... Symbiotic time (s)                     \
                  std  min  ...                std  min   50%     max   
category                    ...                                         
arithmetic        0.8  3.4  ...              280.9  1.5  10.3   705.1   
array             0.0  5.5  ...                4.1  6.3  11.0    15.8   
array_list       60.8  5.5  ...               68.5  4.2  16.8   286.1   
byte_buf         47.3  5.5  ...              162.2  2.1   6.4   879.0   
byte_cursor       3.8  5.5  ...                3.6  2.2   7.2    14.9   
hash_callback     5.5  5.5  ...               62.0  1.2   7.5   111.6   
hash_iter         9.2  5.6  ...             2481.6  6.5  52.9  5000.1   
hash_table       33.3  3.5  ...             2198.3  0.7   3.7  5000.1   
linked_list     209.4  0.0  ...                5.4  0.6   9.3    17.3   
others            0.0  3.5  ...                NaN  4.7   4.7     4.7   
priority_queue  303.4  5.5  ...               20.0  1.4   3.9    76.1   
ring_buffer      19.5  5.5  ...                9.0  1.8   9.5    26.0   
string            1.3  5.5  ...               10.9  1.0   6.2    41.9   

               Klee time (s)                                     
                       count   mean    std   min    50%     max  
category                                                         
arithmetic               6.0    0.9    0.3   0.7    0.8     1.5  
array                    4.0   32.3    6.0  23.6   34.3    37.0  
array_list              23.0   55.4   49.3  13.4   41.3   164.3  
byte_buf                27.0   75.3  124.1  14.1   18.8   619.1  
byte_cursor             17.0   12.8   14.4   0.9    4.5    40.1  
hash_callback            3.0   64.0   45.5  14.7   72.7   104.5  
hash_iter                3.0   20.8    9.7  14.8   15.6    32.0  
hash_table              15.0  104.6  333.4   0.7   14.8  1307.3  
linked_list             18.0    0.7    0.1   0.6    0.7     0.9  
others                   1.0    0.7    NaN   0.7    0.7     0.7  
priority_queue          15.0   46.4   11.6  17.7   50.7    51.3  
ring_buffer              6.0   48.1   26.4  14.2   60.4    73.5  
string                  15.0  139.7  159.7  15.6  105.4   482.3  

[13 rows x 36 columns]

## Part two: Running time per category for CBMC,  SeaHorn and KLEE

The following graph shows the running time results based on each category of jobs.

In [1]:
categories = combined_by_category.groups.keys()
sorted(categories)
YLIMS = 150 # Change the y limits here
for cat in categories:
  # For each category, melt data and draw a line graph with dots
  job_ids = combined_by_category.get_group(cat).index.values
  df = combined_by_category.get_group(cat).drop(columns=['category', 'loc'])
  df = df.rename(columns={"Seahorn normal time (s)": "Seahorn", "CBMC time (s)": "CBMC", "Smack time (s)": "SMACK", "Symbiotic time (s)": "Symbiotic", "Klee time (s)": "KLEE"})
  df = df.melt('job name', var_name='tools',  value_name='time (s)')
  fig, ax = plt.subplots(figsize=(16,4))
  fig.tight_layout()
  # Uncomment the next line if you want to show detailed comparisons 
  # plt.ylim(0, min(df['time (s)'].max(), YLIMS))
  plt.subplots_adjust(hspace = 2.4)
  subg = sns.pointplot(x='job name', y='time (s)', hue='tools', data=df, ax=ax)
  subg.set_title(f'\n\n\n\nFig. {cat}\n',fontsize=21)
  # subg.set_yticklabels(df['time (s)'], fontsize=16)
  subg.set_xlabel("Job Name",fontsize=14)
  subg.set_ylabel("Time (second)",fontsize=14)
  subg.set_xticklabels(df['job name'], rotation='vertical', fontsize=12)
  sns.despine()
  